In [2]:
import os
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import torch
import torchvision
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator
import pandas as pd
import albumentations as A
from albumentations import *
from albumentations.pytorch.transforms import ToTensorV2
import effdet
from effdet import get_efficientdet_config, EfficientDet, DetBenchTrain, create_model_from_config
from effdet.efficientdet import HeadNet

from datetime import datetime
import time
import random
import cv2
from sklearn.model_selection import StratifiedKFold
from torch.utils.data import Dataset,DataLoader
from torch.utils.data.sampler import SequentialSampler, RandomSampler
from torch.cuda.amp import GradScaler, autocast
from glob import glob

In [3]:
def seed_everything(seed):
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
seed_everything(0)

In [4]:
labelmap = {'fruit_healthy':1, 'fruit_woodiness':2, 'fruit_brownspot':3}
labelmap_inv = {k:v for v,k in labelmap.items()}

class PassionFruitDatasetTest(torch.utils.data.Dataset):
    def __init__(self, csv_path, img_path, transforms):
        self.csv = pd.read_csv(csv_path)
        self.root = img_path
        self.transforms = transforms

    def __getitem__(self, idx):
        # load images and masks
        row = self.csv.iloc[idx]
        img_path = os.path.join(self.root, row.Image_ID) + '.jpg'
        img = np.array(Image.open(img_path).convert("RGB")).astype('uint8')
        if self.transforms is not None:
            annotations = {'image': img}
            augmented = self.transforms(**annotations)
            img = augmented['image']
        
        return img, row.Image_ID

    def __len__(self):
        return len(self.csv)

In [5]:
def get_net(model_name):
    config = get_efficientdet_config(model_name)
    config.num_classes = 3
    config.pretrained = True
    config.image_size = [512,512]
    config.label_smoothing = 0.1
    
    net = create_model_from_config(config)
    
    return DetBenchPredict(net)


In [6]:
transform_test = Compose([
    Normalize(),
    ToTensorV2(),
])

val_ds = PassionFruitDatasetTest(
    'data/Test.csv',
    'data/Test_Images/Test_Images', 
    transform_test
)

data_loader = DataLoader(
    val_ds,
    batch_size=2,
    shuffle=False,
    num_workers=0,
    drop_last=False
)

In [7]:
net = get_net('tf_efficientdet_d4')
checkpoint = torch.load('logs/tf_effdet4-heavyaug-pseudo/last-checkpoint.bin')
net.model.load_state_dict(checkpoint['model_state_dict'])

<All keys matched successfully>

In [8]:
def make_predictions(images, score_threshold=0.2):
    images = images.cuda().float()
    predictions = []
    net.cuda()
    with torch.no_grad():
        det = net(images, {'img_size':(torch.ones((images.shape[0], 2)) * 512).float().cuda(),'img_scale':torch.ones(images.shape[0]).float().cuda()})
        for i in range(images.shape[0]):
            boxes = det[i].detach().cpu().numpy()[:,:4]    
            scores = det[i].detach().cpu().numpy()[:,4]
            classes = det[i].detach().cpu().numpy()[:,5]
            indexes = np.where(scores > score_threshold)[0]
            boxes = boxes[indexes]
            
            
            predictions.append({ 
                'boxes': boxes[indexes],
                'scores': scores[indexes],
                'classes': classes[indexes]
            })
    return [predictions]

def iou(box):
    mat = []
    ious = []
    for a in range(len(box)):
        for b in range(len(box)):
            if a != b:
                boxA = box[a]
                boxB = box[b]
                xA = max(boxA[0], boxB[0])
                yA = max(boxA[1], boxB[1])
                xB = min(boxA[2], boxB[2])
                yB = min(boxA[3], boxB[3])
                interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
                boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
                boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
                iou = interArea / float(boxAArea + boxBArea - interArea)
                if iou not in ious:
                    ious.append(iou)
                    mat.append([a,b])
    return np.array(ious), np.array(mat)

def postprocessing(boxes, scores, labels):
    boxes = (boxes).astype(np.int32).clip(min=0, max=512)
    ious, mat = iou(boxes)
    del_index = []
    for i in mat[ious > 0.8]:
        if scores[i[0]] > scores[i[1]]:
            del_index.append(i[1])
        else:
            del_index.append(i[0])
    boxes = np.delete(boxes,del_index,axis = 0)
    scores = np.delete(scores,del_index)
    labels = np.delete(labels,del_index)
    
    return boxes, scores, labels

In [9]:
results = []

for images, image_id in data_loader:
    predictions = make_predictions(images, 0.1)
    
    for i, image in enumerate(images):
        boxes, scores, labels = predictions[0][i]['boxes'], predictions[0][i]['scores'], predictions[0][i]['classes']
        
        boxes, scores, labels = postprocessing(boxes, scores, labels)
        result = {
            'image_id': image_id[i],
            'boxes': boxes,
            'labels': labels,
            'scores': scores,
        }
        results.append(result)

C:\Users\chenp\anaconda3\envs\ml\lib\site-packages\effdet\bench.py:45: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  indices_all = cls_topk_indices_all // num_classes


In [10]:
train = pd.read_csv('data/labels_train.csv')
train_max = max(train.image_id)

In [11]:
train_ = pd.DataFrame(columns = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax', 'image_id','folder'])

img_name = ''
counter = train_max + 1
counter2 = 0

tr = 0.2
for i in results:
    ID = i['image_id']
    boxes = i['boxes']
    scores = i['scores']
    
    mask = scores > tr
    if not mask.any():
        mask[np.argmax(scores)] = True
    labels = i['labels'][mask].tolist()
    boxes = i['boxes'][mask]
    labels = [labelmap_inv[j] for j in i['labels']]
    
    for j, box in enumerate(boxes):
       
        train_.loc[counter2] = [f'{ID}.jpg',
                         512,
                         512,
                         labels[j],
                         box[0],
                         box[1],
                         box[2],
                         box[3],
                         counter,
                        'Test']
        
        counter2 += 1
    counter += 1
    
train_ = pd.concat((train,train_))

In [12]:
train_.to_csv('data/labels_pseudo_train.csv')

In [13]:
len(train_)

5206